# First Data preprocessing and cleaning

In [17]:
import numpy as np

In [1]:
data_dir = "C:/Users/Thibault/Documents/Data/Hackathon_sante/data"

In [2]:
f = open(data_dir + "/training_30")

In [3]:
# Reading file
lines = f.readlines()

In [14]:
lines = [l.replace("\n","") for l in lines]
lines[27:27+3]

['2342.7\t2429.1\t2536.5\t2640.6\t2749.3\t2812.\t2758.3\t2810.5\t2701.2\t2736.9\t2647.5\t2731.3\t2681.5\t2750.2\t2695.5\t2784.7\t2744.6\t2797.9\t2707.3\t2745.7\t2729.6\t2820.7\t2818.2\t2876.\t2869.4\t2921.3\t2821.5\t2801.8\t2708.4\t2749.8\t2739.6\t2777.\t2709.3\t2782.4\t2727.2\t2726.2\t2776.\t2779.7\t2915.4\t2916.5\t3040.9\t3069.4\t3191.1\t3234.6\t3365.8\t3375.1\t3497.8\t3612.4\t3578.5\t3679.2\t3727.8\t3847.5\t3901.5\t3826.2\t3830.2\t3761.5\t3805.9\t3936.7\t3990.9\t3934.9\t3925.7\t3871.7\t3721.\t3655.9\t3512.1\t3483.3\t3560.6\t3542.3\t3575.\t3682.4\t3719.1\t3613.4\t3559.5\t3457.1\t3327.5\t3294.2\t3173.5\t3186.5\t3303.6\t3322.3\t3313.9\t3361.3\t3357.3\t3212.2\t3129.\t2988.2\t2914.2\t2955.\t2893.7\t2927.\t3051.9\t3092.9\t3012.9\t3003.5\t2929.9\t2784.5\t2723.2\t2583.9\t2562.2\t2644.7\t2632.1\t2660.8\t2763.3\t2797.9\t2685.7\t2634.4\t2526.6\t2402.4\t2370.\t2253.\t2285.4\t2407.6\t2453.3\t2438.3\t2475.6\t2470.4\t2611.4\t2626.8\t2758.\t2811.4\t2943.3\t3009.\t3155.1\t3214.4\t3352.9\t3451.5\t342

In [81]:
def coords_split(lister, splice):
    # Split all passed sequences by "splice" and return an array of them
    # Convert string fragments to float 
    coords = []
    for c in lister:
        coords.append([float(a) for a in c.split(splice)])
    return coords

In [82]:
np.array(coords_split(lines[27:27+3], "\t")).shape

(3, 921)

In [86]:
# Scan first n proteins
names = []
seqs = []
coords = []
pssms = []

# Record names, seqs and coords for each protein btwn 1-n
for i in range(0,len(lines)):
    if len(coords) == 995:
        break    
    # Start recording
    if lines[i] == "[ID]":
        names.append(lines[i+1]) 
    elif lines[i] == "[PRIMARY]":
        seqs.append(lines[i+1])
    elif lines[i] == "[TERTIARY]":
        coords.append(coords_split(lines[i+1:i+4], "\t"))
    elif lines[i] == "[EVOLUTIONARY]":
        pssms.append(coords_split(lines[i+1:i+21], "\t"))
        # Progress control
        if len(names)%50 == 0:
            print("Currently @ ", len(names), " out of n")

Currently @  50  out of n
Currently @  100  out of n
Currently @  150  out of n
Currently @  200  out of n
Currently @  250  out of n
Currently @  300  out of n
Currently @  350  out of n
Currently @  400  out of n
Currently @  450  out of n
Currently @  500  out of n
Currently @  550  out of n
Currently @  600  out of n
Currently @  650  out of n
Currently @  700  out of n
Currently @  750  out of n
Currently @  800  out of n
Currently @  850  out of n
Currently @  900  out of n
Currently @  950  out of n


In [92]:
# Could use "Using LinearAlgebra + built-in norm()" but gotta learn Julia
def norm(vector):
    return np.sqrt(np.sum([v*v for v in vector]))

In [93]:
print("Total number of proteins: ", len(seqs))

n = 200
under = []
for i in range(0,len(seqs)):
    if len(seqs[i])<200:
        under.append(i)
        # println("Seelected with: ", length(seqs[i]), " number: ", i)

print("Number of proteins under ", n, " : ", len(under))

Total number of proteins:  995
Number of proteins under  200  :  636


In [99]:
dists = []
# Get distances btwn pairs of AAs - only for prots under 200
for k in under:
    # Get distances from coordinates
    dist = []
    for i in range(0,len(coords[k][0])):
        # Only pick coords for C-alpha carbons! - position (1/3 of total data)
        # i%3 == 2 Because juia arrays start at 1 - Python: i%3 == 1
        if i%3 == 1:
            aad = [] # Distance to every AA from a given AA
            for j in range(0,len(coords[k][0])):
                if j%3 == 1:
                    
                    aad.append(norm([coords[k][0][i] - coords[k][0][j]
                                     ,coords[k][1][i] - coords[k][1][j]
                                     ,coords[k][2][i] - coords[k][2][j]]))
            dist.append(aad)
    dists.append(dist)
    
    # Progress control
    if len(dists)%50 == 0:
        print("Dists Currently @ ", len(dists), " out of n (500)")

Dists Currently @  50  out of n (500)
Dists Currently @  100  out of n (500)
Dists Currently @  150  out of n (500)
Dists Currently @  200  out of n (500)
Dists Currently @  250  out of n (500)
Dists Currently @  300  out of n (500)
Dists Currently @  350  out of n (500)
Dists Currently @  400  out of n (500)
Dists Currently @  450  out of n (500)
Dists Currently @  500  out of n (500)
Dists Currently @  550  out of n (500)
Dists Currently @  600  out of n (500)


In [100]:
# Check everything's alright
n = 1
print("id: ", names[n])
print("seq: ", seqs[n]) 
print("sample coord: ", coords[n][1-1][1-1]) 
print("sample dist: ", dists[n][1-1][5-1])

id:  2EUL_d2euld1
seq:  MAREVKLTKAGYERLMQQLERERERLQEATKILQELMESSDDYDDSGLEAAKQEKARIEARIDSLEDILSRAVILEE
sample coord:  981.8
sample dist:  2674.450579090965


In [106]:
len(coords)

995

In [107]:
len(dists)

636

In [126]:
len(pssms[2][0])

142

In [140]:
# Data is OK. Save it to a file.
with open(data_dir + "/full_under_200.txt", "w+") as f:
    aux = [0]
    for k in under:
        aux.append(aux[len(aux)-1]+1-1)
        # ID
        f.write("\n[ID]\n")
        f.write(names[k])
        # Seq
        f.write("\n[PRIMARY]\n")
        f.write(seqs[k])
        # PSSMS
        f.write("\n[EVOLUTIONARY]\n")
        f.write(",".join([str(val) for val in pssm]))
#         for pssm in pssms[k]:
#             for pssm_ in pssm:
#                 f.write(str(pssm_))
#                 if 
#                 f.write(",")
        # Coords
        f.write("\n[TERTIARY]\n")
        f.write(",".join([str(val) for val in coords]))
#         for coord in coords[k]:
#             f.write(str(coord))
#             f.write(",")
        # Dists
        f.write("\n[DIST]\n")
        # Check that saved proteins are less than 200 AAs
        if len(dists[aux[len(aux)-1]][1-1])>200:
            print("error when checking protein in dists n: ", aux[len(aux)], " length: ", length(dists[aux[len(aux)]][1-1]))
            break
        else:
            f.write(",".join([str(v) for val in dist for v in dist]))
#             for dist in dists[aux[len(aux)-1]]:
#                 for dist_ in dist:
#                     f.write(dist_)
#                     f.write(",")

# Done!